Tasks:
- Download the Credit.csv dataset from ISL: https://www.statlearning.com/s/Credit.csv
- Preprocess the dataset as you see fit.

In [119]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from pygam import LinearGAM, s

# 1. Load the dataset
df = pd.read_csv("https://www.statlearning.com/s/Credit.csv")

# 2. inspect the dataframe
print(df.head())
print(df.info())
print(df.describe())
df.isnull().sum() 


    Income  Limit  Rating  Cards  Age  Education  Own Student Married Region  \
0   14.891   3606     283      2   34         11   No      No     Yes  South   
1  106.025   6645     483      3   82         15  Yes     Yes     Yes   West   
2  104.593   7075     514      4   71         11   No      No      No   West   
3  148.924   9504     681      3   36         11  Yes      No      No   West   
4   55.882   4897     357      2   68         16   No      No     Yes  South   

   Balance  
0      333  
1      903  
2      580  
3      964  
4      331  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Income     400 non-null    float64
 1   Limit      400 non-null    int64  
 2   Rating     400 non-null    int64  
 3   Cards      400 non-null    int64  
 4   Age        400 non-null    int64  
 5   Education  400 non-null    int64  
 6   Own        

Income       0
Limit        0
Rating       0
Cards        0
Age          0
Education    0
Own          0
Student      0
Married      0
Region       0
Balance      0
dtype: int64

In [120]:
# 3: prepare the dataset
predictors = ['Limit', 'Rating', 'Cards', 'Age', "Education", "Own", "Student", "Married", "Region"]
target = 'Balance'
df = df[predictors + [target]]
df = df.dropna()
df = pd.get_dummies(df, columns=["Region"])
df['Own'] = df['Own'].map({'No': 0, 'Yes': 1})
df['Student'] = df['Student'].map({'No': 0, 'Yes': 1})
df['Married'] = df['Married'].map({'No': 0, 'Yes': 1})
df.head()

,Limit,Rating,Cards,Age,Education,Own,Student,Married,Balance,Region_East,Region_South,Region_West
0,3606,283,2,34,11,0,0,1,333,False,True,False
1,6645,483,3,82,15,1,1,1,903,False,False,True
2,7075,514,4,71,11,0,0,0,580,False,False,True
3,9504,681,3,36,11,1,0,0,964,False,False,True
4,4897,357,2,68,16,0,0,1,331,False,True,False


- Fit non-linear models (GLM, GAM, RF, XGBoost) to predict Balance from the other variables in the dataset. You are not allowed to use the Income variable as a predictor, due to its high collinearity with Balance. Whether you fit one or more types of models is up to you, the same goes for the choice of hyperparameters and how you find them. However, you should aim for achieving good predictive performance, and being relatively certain the performance holds up in new data. In other words, using a train/validation/test-split or a (nested) cross-validation is probably a good idea. When you are satisfied, report model performance in a comprehensible manner.

In [121]:
# split the dataset into training and testing set
np.random.seed(42)
df = df.sample(frac=1.)
train = df[:int(len(df) * 0.8)].astype(float)
test = df[int(len(df) * 0.8):].astype(float)

train_X = train[['Limit', 'Rating', 'Cards', 'Age', "Education", "Own", "Student", "Married", "Region_East", "Region_South", "Region_West"]]
test_X = test[['Limit', 'Rating', 'Cards', 'Age', "Education", "Own", "Student", "Married", "Region_East", "Region_South", "Region_West"]]

train_y = train["Balance"]
test_y = test["Balance"]

X = df[['Limit', 'Rating', 'Cards', 'Age', "Education", "Own", "Student", "Married", "Region_East", "Region_South", "Region_West"]].astype(float).values
y = df["Balance"].values


In [122]:
# ------------------ GLM (no CV) ------------------
import statsmodels.api as sm
link = sm.genmod.families.links.Log()
glm_model = sm.GLM(train_y, train_X, family=sm.families.Gamma(link=link))
glm_result = glm_model.fit()
y_pred_glm = glm_result.predict(test_X)

# ------------------ GAM with 10-Fold CV ------------------
from sklearn.model_selection import KFold, cross_val_score
from pygam import LinearGAM, s
kf = KFold(n_splits=10, shuffle=True, random_state=42)
gam_rmse = []

for train_idx, val_idx in kf.split(X):
    gam = LinearGAM(s(0) + s(1) + s(2) + s(3) + s(4) +
                    s(5) + s(6) + s(7) + s(8) + s(9) + s(10))
    gam.fit(X[train_idx], y[train_idx])
    preds = gam.predict(X[val_idx])
    rmse = mean_squared_error(y[val_idx], preds, squared=False)
    gam_rmse.append(rmse)

# Final GAM fit for test predictions
gam_final = LinearGAM(s(0) + s(1) + s(2) + s(3) + s(4) +
                      s(5) + s(6) + s(7) + s(8) + s(9) + s(10))
gam_final.fit(train_X.values, train_y.values)
y_pred_gam = gam_final.predict(test_X.values)

# ------------------ Random Forest with 10-Fold CV ------------------
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf_rmse = cross_val_score(rf, X, y, cv=kf,
                          scoring=make_scorer(mean_squared_error, squared=False))
rf.fit(train_X, train_y)
y_pred_rf = rf.predict(test_X)

# ------------------ XGBoost with 10-Fold CV ------------------
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_rmse = cross_val_score(xgb, X, y, cv=kf,
                           scoring=make_scorer(mean_squared_error, squared=False))
xgb.fit(train_X, train_y)
y_pred_xgb = xgb.predict(test_X)

# ------------------ Evaluation ------------------
def report(name, y_true, y_pred):
    print(f"{name:12s} | RMSE: {mean_squared_error(y_true, y_pred, squared=False):7.2f} "
          f"| MAE: {mean_absolute_error(y_true, y_pred):7.2f} "
          f"| R²: {r2_score(y_true, y_pred):.3f}")

print("\nModel Performance on Test Set:")
print("-" * 50)
report("GLM", test_y, y_pred_glm)
report("GAM", test_y, y_pred_gam)
report("Random Forest", test_y, y_pred_rf)
report("XGBoost", test_y, y_pred_xgb)

# ------------------ CV Summary ------------------
print("\nCross-Validation RMSE (10-Fold):")
print("-" * 50)
print(f"GAM           | CV RMSE: {np.mean(gam_rmse):7.2f} ± {np.std(gam_rmse):.2f}")
print(f"Random Forest | CV RMSE: {np.mean(rf_rmse):7.2f} ± {np.std(rf_rmse):.2f}")
print(f"XGBoost       | CV RMSE: {np.mean(xgb_rmse):7.2f} ± {np.std(xgb_rmse):.2f}")

/Users/qiqi/miniforge3/envs/python-for-scicomp/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/qiqi/miniforge3/envs/python-for-scicomp/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/qiqi/miniforge3/envs/python-for-scicomp/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/qiqi/miniforge3/envs/python-for-scicomp/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: 


Model Performance on Test Set:
--------------------------------------------------
GLM          | RMSE: 5619.46 | MAE: 1678.26 | R²: -134.878
GAM          | RMSE:  178.81 | MAE:  138.98 | R²: 0.862
Random Forest | RMSE:  196.84 | MAE:  145.47 | R²: 0.833
XGBoost      | RMSE:  192.39 | MAE:  143.36 | R²: 0.841

Cross-Validation RMSE (10-Fold):
--------------------------------------------------
GAM           | CV RMSE:  188.89 ± 32.41
Random Forest | CV RMSE:  186.83 ± 32.68
XGBoost       | CV RMSE:  194.62 ± 31.65


/Users/qiqi/miniforge3/envs/python-for-scicomp/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/qiqi/miniforge3/envs/python-for-scicomp/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/qiqi/miniforge3/envs/python-for-scicomp/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/qiqi/miniforge3/envs/python-for-scicomp/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: 

- Reflect on the choices you made: Why did you choose the model(s) you did? What about the hyperparameters? How did you tune them?

GLM(with loglink), GAM, Random Forest, and XGBoost models were fitted to capture the non-linear relationships between the features in the data. I would prefer the XGboost because it usually provides the best performance. But also wanted to compare these models so all of them were fitted and three of them performed a 10 fold cross validation.
GLM is appropriate for modeling continuous, positive response variables. However, it assumes a log linear relationship, which may not be flexible enough for our data. GAM captures non-linear relationships by applying smooth splines to each feature, and it balances the interpretability and flexibility. However it does not allow for the interactions between predictors. Random Forest is a non-parametric model that automatically handles feature interactions and non-linearities. XGBoost is a good selection because it builds decision trees sequentially based on the corrections of errors of the previous ones, which givees it a high performance. For the hyperparameter tuning I used the default settings. 


- Reflect upon the performance of the model(s): How well did it/they perform? What would you expect the performance to be on new data? How can you argue this is the case? What other choices could have made you more or less certain about this?

GLM performed very poorly, with a large MAE and a negative R squared. This confirms that the linear log-link assumption was too restrictive for this task, and it failed to capture the structure in the data. GAM performed the best overall with the lowest MAE. Not sure why here GAM achieved the best performance than the XGboost. This suggests that the relationship between predictors and Balance is non-linear but can be captured through additive smooth effects? Random Forest and XGBoost also performed well with relatively low MAE. XGBoost slightly outperformed Random Forest, likely due to its boosting mechanism correcting residual errors. Given the results from CV and model performance, I would expect GAM and Random Forest to generalize well to new data due to their relatively low and stable RMSEs. More parameter tunings probably is needed for RF and XGBoost.

